# Problema de clasificación
En este ejercicio se usará un dataset de la temperatura en Australia, en el que el objetivo es predecir si lloverá el día de mañana. En este conjunto de datos tenemos varios tipos de datos; entre estos la temperatura, cantidad de lluvia, evaporación, etc. En total conforman un conjunto de 23 columnas.


In [2]:
import os 
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn
from sklearn.datasets import make_blobs
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV
import random
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV 
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import cross_val_score
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold

In [3]:
data = pd.read_csv('weatherAUS.csv')

In [4]:
data.head()

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,2008-12-01,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,W,...,71.0,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,No,No
1,2008-12-02,Albury,7.4,25.1,0.0,NaN,NaN,WNW,44.0,NNW,...,44.0,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3,No,No
2,2008-12-03,Albury,12.9,25.7,0.0,NaN,NaN,WSW,46.0,W,...,38.0,30.0,1007.6,1008.7,NaN,2.0,21.0,23.2,No,No
3,2008-12-04,Albury,9.2,28.0,0.0,NaN,NaN,NE,24.0,SE,...,45.0,16.0,1017.6,1012.8,NaN,NaN,18.1,26.5,No,No
4,2008-12-05,Albury,17.5,32.3,1.0,NaN,NaN,W,41.0,ENE,...,82.0,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,No,No


## Preprocesamiento
En esta seccion, vamos a proceder a limpiar el dataset

In [5]:
data.columns

Index(['Date', 'Location', 'MinTemp', 'MaxTemp', 'Rainfall', 'Evaporation',
       'Sunshine', 'WindGustDir', 'WindGustSpeed', 'WindDir9am', 'WindDir3pm',
       'WindSpeed9am', 'WindSpeed3pm', 'Humidity9am', 'Humidity3pm',
       'Pressure9am', 'Pressure3pm', 'Cloud9am', 'Cloud3pm', 'Temp9am',
       'Temp3pm', 'RainToday', 'RainTomorrow'],
      dtype='object')

In [6]:
data.shape

(145460, 23)

In [7]:
data.describe()

,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm
count,143975.000000,144199.000000,142199.000000,82670.000000,75625.000000,135197.000000,143693.000000,142398.000000,142806.000000,140953.000000,130395.00000,130432.000000,89572.000000,86102.000000,143693.000000,141851.00000
mean,12.194034,23.221348,2.360918,5.468232,7.611178,40.035230,14.043426,18.662657,68.880831,51.539116,1017.64994,1015.255889,4.447461,4.509930,16.990631,21.68339
std,6.398495,7.119049,8.478060,4.193704,3.785483,13.607062,8.915375,8.809800,19.029164,20.795902,7.10653,7.037414,2.887159,2.720357,6.488753,6.93665
min,-8.500000,-4.800000,0.000000,0.000000,0.000000,6.000000,0.000000,0.000000,0.000000,0.000000,980.50000,977.100000,0.000000,0.000000,-7.200000,-5.40000
25%,7.600000,17.900000,0.000000,2.600000,4.800000,31.000000,7.000000,13.000000,57.000000,37.000000,1012.90000,1010.400000,1.000000,2.000000,12.300000,16.60000
50%,12.000000,22.600000,0.000000,4.800000,8.400000,39.000000,13.000000,19.000000,70.000000,52.000000,1017.60000,1015.200000,5.000000,5.000000,16.700000,21.10000
75%,16.900000,28.200000,0.800000,7.400000,10.600000,48.000000,19.000000,24.000000,83.000000,66.000000,1022.40000,1020.000000,7.000000,7.000000,21.600000,26.40000
max,33.900000,48.100000,371.000000,145.000000,14.500000,135.000000,130.000000,87.000000,100.000000,100.000000,1041.00000,1039.600000,9.000000,9.000000,40.200000,46.70000


#### Se verifican los datos nulos

In [8]:
data.nunique()

Date             3436
Location           49
MinTemp           389
MaxTemp           505
Rainfall          681
Evaporation       358
Sunshine          145
WindGustDir        16
WindGustSpeed      67
WindDir9am         16
WindDir3pm         16
WindSpeed9am       43
WindSpeed3pm       44
Humidity9am       101
Humidity3pm       101
Pressure9am       546
Pressure3pm       549
Cloud9am           10
Cloud3pm           10
Temp9am           441
Temp3pm           502
RainToday           2
RainTomorrow        2
dtype: int64

#### Se borran los datos nulos

In [9]:
data = data.dropna()

In [10]:
data.shape

(56420, 23)

In [11]:
data.isnull().sum()

Date             0
Location         0
MinTemp          0
MaxTemp          0
Rainfall         0
Evaporation      0
Sunshine         0
WindGustDir      0
WindGustSpeed    0
WindDir9am       0
WindDir3pm       0
WindSpeed9am     0
WindSpeed3pm     0
Humidity9am      0
Humidity3pm      0
Pressure9am      0
Pressure3pm      0
Cloud9am         0
Cloud3pm         0
Temp9am          0
Temp3pm          0
RainToday        0
RainTomorrow     0
dtype: int64

#### Se verifica que exista representatividad para cada tipo (llueve o no llueve) 

In [12]:
data.head()
data.RainTomorrow.value_counts()

No     43993
Yes    12427
Name: RainTomorrow, dtype: int64

In [13]:
data.shape

(56420, 23)

In [14]:
y = data.RainTomorrow
x = data.drop(['Date','RainToday', 'Location',  'WindGustDir', 'WindDir9am','RainTomorrow', 'WindDir3pm', 'Rainfall'], axis=1)

x.head()

,MinTemp,MaxTemp,Evaporation,Sunshine,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm
6049,17.9,35.2,12.0,12.3,48.0,6.0,20.0,20.0,13.0,1006.3,1004.4,2.0,5.0,26.6,33.4
6050,18.4,28.9,14.8,13.0,37.0,19.0,19.0,30.0,8.0,1012.9,1012.1,1.0,1.0,20.3,27.0
6052,19.4,37.6,10.8,10.6,46.0,30.0,15.0,42.0,22.0,1012.3,1009.2,1.0,6.0,28.7,34.9
6053,21.9,38.4,11.4,12.2,31.0,6.0,6.0,37.0,22.0,1012.7,1009.1,1.0,5.0,29.1,35.6
6054,24.2,41.0,11.2,8.4,35.0,17.0,13.0,19.0,15.0,1010.7,1007.4,1.0,6.0,33.6,37.6


In [15]:
x.shape

(56420, 15)

# Reducción  de dimensionalidad con PCA

### Primero escalamos el dataset

In [16]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(x)

StandardScaler()

In [17]:
scaled_data=scaler.transform(x)
print(scaled_data)

[[ 0.69120848  1.57529783  1.75769102 ...  0.25441126  1.27818584
   1.56362087]
 [ 0.76913098  0.67150378  2.51521564 ... -1.25660337  0.31897996
   0.62746694]
 [ 0.92497598  1.91960032  1.43303762 ...  0.63216492  1.59792114
   1.78303195]
 ...
 [ 1.12757448  1.23099533  0.02620618 ... -1.63435703  1.00412702
   1.37346461]
 [ 0.94056048  1.08753596  0.18853289 ... -1.25660337  1.00412702
   0.94926986]
 [ 1.04965198  1.07319002  0.02620618 ...  0.25441126  1.09547996
   1.21256315]]


### Luego aplicamos PCA

Buscamos que tenga la cantidad mínima  de features, con un 95% de varianza.

In [18]:
from sklearn.decomposition import PCA
#pca=PCA(n_components=2)

pca=PCA(.95)
principalComponents = pca.fit(scaled_data)
pca_x = pca.transform(scaled_data)

In [19]:
x = pca_x
x.shape

(56420, 9)

In [20]:
from sklearn.model_selection import train_test_split
train_x, test_x, train_y, test_y = train_test_split(x, y, test_size= 0.2)

In [21]:
print(train_x.shape)

print(train_y.shape)

(45136, 9)
(45136,)


# Neural Network

# Red neuronal compleja
Primero realizamos una red neuronal más compleja; con varias capas ocultas y distintas cantidades de neuronas. 
La capa de entrada, contiene 9 neuronas,una por cada parámetro. 

Para la cantidad de neuronas en las capas ocultas usamos una regla del pulgar, que es la siguiente
$$ N_h=\dfrac{N_s}{\alpha \cdot (N_i+N_o)}$$

En la que 
* $N_i =$ Número de neuronas en la capa de entrada (9)
* $N_o =$ Número de neuronas en la capa de salida (1)
* $N_s =$ Número de muestras en el _training_set_ (45136)
* $\alpha =$ Factor de escala arbitrario, usualmente entre 2-10 (5)

En este caso tenemos 9 neuronas de entrada, por cada _feature_, una neurona de salida. Nuestro _training_set_ se compone de 45136 datos, y para el $\alpha$ elegí el valor más alto; 10, para evitar la menor cantidad de neuronas.  

$$ N_h=\dfrac{N_s}{\alpha \cdot (N_i+N_o)} = \dfrac{45136}{10 \cdot (9+1)}= 451.36$$

Por efectos de recursos computacionales usaré otra regla, dado que 451 neuronas es una cantidad muy grande para mi computadora. 
En cambio usare una regla que establece que que la cantidad de neuronas debe ser menor que el doble de las neuronas en la capa de entrada. Por lo que debe ser menor a 18 neuronas, en mi caso selecciones 10 neuronas



## Prueba de parámetros

## Batch size y epochs

En esta sección estaremos probando con distintas cantidades de batch y epochs, para averiguar cual es la que tiene mejores resultados.


In [22]:
# Function to create model,for KerasClassifier
def create_complex_model():
    #defining my model
    mymodel = Sequential()
    mymodel.add(Dense(9, input_dim=9, activation='relu'))
    mymodel.add(Dense(27, activation='relu'))
    mymodel.add(Dense(12, activation='relu'))
    mymodel.add(Dense(1, activation='sigmoid'))
    
    # Compile the model
    mymodel.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return mymodel

In [83]:
# create model
model = KerasClassifier(build_fn=create_complex_model)

### Parameters
En esta parte vamos a probar la cantidad óptima de _epochs_ y _batch_. 
* batch_size, determina la cantidad de muestras que pasan por la red en un _epoch_
* epochs, la cantidad de veces que pasan datos por la red.



In [23]:
# define the grid search parameters
batchSize = [10, 50,  100]
epochs = [10, 30, 50]
grid = dict(batch_size=batchSize, epochs=epochs)

### Tuning Hyperparameters

 Se ejecuta la combinación de cada parámetro en un stratified cross validation, con el objetivo de evitar algún sesgo producido por la distribución de los parámetros.


In [24]:
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
grid_search = GridSearchCV(estimator=model, param_grid=grid, n_jobs=-1, cv=cv, scoring='accuracy',error_score=0)
grid_result = grid_search.fit(train_x, train_y)

Epoch 1/50
4514/4514 [==============================] - 4s 743us/step - loss: 0.3728 - accuracy: 0.8335
Epoch 2/50
4514/4514 [==============================] - 3s 734us/step - loss: 0.3543 - accuracy: 0.8433
Epoch 3/50
4514/4514 [==============================] - 3s 734us/step - loss: 0.3516 - accuracy: 0.8445
Epoch 4/50
4514/4514 [==============================] - 3s 731us/step - loss: 0.3504 - accuracy: 0.8457
Epoch 5/50
4514/4514 [==============================] - 3s 714us/step - loss: 0.3493 - accuracy: 0.8450
Epoch 6/50
4514/4514 [==============================] - 3s 747us/step - loss: 0.3487 - accuracy: 0.8449
Epoch 7/50
4514/4514 [==============================] - 4s 796us/step - loss: 0.3480 - accuracy: 0.8464
Epoch 8/50
4514/4514 [==============================] - 3s 748us/step - loss: 0.3475 - accuracy: 0.8459
Epoch 9/50
4514/4514 [==============================] - 3s 738us/step - loss: 0.3470 - accuracy: 0.8468
Epoch 10/50
4514/4514 [==============================] - 3s 750u

### Best Hyperparameters

Imprimimos la combinación de los parámetros y sus resultados, obtenemos la mejor combinación de estos para obtener el mejor resultado.

In [25]:
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.845452 using {'batch_size': 10, 'epochs': 50}
0.845445 (0.003797) with: {'batch_size': 10, 'epochs': 10}
0.845053 (0.003869) with: {'batch_size': 10, 'epochs': 30}
0.845452 (0.003698) with: {'batch_size': 10, 'epochs': 50}
0.845186 (0.003681) with: {'batch_size': 50, 'epochs': 10}
0.845437 (0.003508) with: {'batch_size': 50, 'epochs': 30}
0.844455 (0.004745) with: {'batch_size': 50, 'epochs': 50}
0.843547 (0.004155) with: {'batch_size': 100, 'epochs': 10}
0.844943 (0.004349) with: {'batch_size': 100, 'epochs': 30}
0.844647 (0.003810) with: {'batch_size': 100, 'epochs': 50}


Como se puede observar la precision, alterando epocas y batch_size no cambia de manera significativa. Sin embargo podemos ver que la que obtuvo mejor precision du la de 50 epocs con un batch size de 10.

In [23]:
batch_model = KerasClassifier(build_fn=create_complex_model, epochs=50, batch_size=10, verbose=0)


### Resultados con los mejores parámetros

Para evitar que haya algún tipo de sesgo en los datos se realizaron pruebas 

In [24]:

train_x, test_x, train_y, test_y = train_test_split(x, y, test_size= 0.2, random_state= random.randint(0, 100))
batch_model.fit(train_x, train_y, verbose=0)
probabilities = batch_model.predict_proba(test_x)


Epoch 1/50
4514/4514 [==============================] - 4s 712us/step - loss: 0.3645 - accuracy: 0.8397
Epoch 2/50
4514/4514 [==============================] - 3s 726us/step - loss: 0.3514 - accuracy: 0.8445
Epoch 3/50
4514/4514 [==============================] - 4s 829us/step - loss: 0.3496 - accuracy: 0.8452
Epoch 4/50
4514/4514 [==============================] - 3s 684us/step - loss: 0.3481 - accuracy: 0.8453
Epoch 5/50
4514/4514 [==============================] - 3s 678us/step - loss: 0.3472 - accuracy: 0.8460
Epoch 6/50
4514/4514 [==============================] - 3s 678us/step - loss: 0.3464 - accuracy: 0.8464
Epoch 7/50
4514/4514 [==============================] - 3s 679us/step - loss: 0.3460 - accuracy: 0.8463
Epoch 8/50
4514/4514 [==============================] - 3s 681us/step - loss: 0.3452 - accuracy: 0.8454
Epoch 9/50
4514/4514 [==============================] - 3s 684us/step - loss: 0.3448 - accuracy: 0.8472
Epoch 10/50
4514/4514 [==============================] - 3s 678u

In [25]:
y = data.RainTomorrow
kfold = KFold(n_splits=10, shuffle=True, random_state=random.randint(0, 10)) 
results = cross_val_score(batch_model, x, y, cv=kfold, n_jobs=-1)
print(results.mean())

0.8452144622802734


## Función de activación

En esta sección se realizarán pruebas al modelo con distintas funciones de activación, esto con el propósito de conocer cuál es la más óptima para este tipo de datos.


In [22]:
def create_model_activation(init_mode='uniform'):
    #defining my model
    mymodel = Sequential()
    mymodel.add(Dense(9, kernel_initializer=init_mode, input_dim=9, activation='relu'))
    mymodel.add(Dense(27, kernel_initializer=init_mode, activation='relu'))
    mymodel.add(Dense(12, kernel_initializer=init_mode,activation='relu'))
    mymodel.add(Dense(1, activation='sigmoid'))
    
    # Compile the model
    mymodel.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return mymodel

### Creacion del modelo

Creamos el modelo de nuevo, con la mejor combinacino de epochs y batch

In [ ]:
model = KerasClassifier(build_fn=create_model_activation, epochs=50, batch_size=10)

### Parameters
En esta parte vamos a probar cual es la mejor funcion de activacion


In [36]:
init_mode = ['uniform', 'lecun_uniform', 'normal', 'zero', 'glorot_normal', 'glorot_uniform', 'he_normal', 'he_uniform']
param_grid = dict(init_mode=init_mode)

In [37]:
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(train_x, train_y)


Epoch 1/50
4514/4514 [==============================] - 4s 678us/step - loss: 0.3695 - accuracy: 0.8364
Epoch 2/50
4514/4514 [==============================] - 3s 673us/step - loss: 0.3554 - accuracy: 0.8428
Epoch 3/50
4514/4514 [==============================] - 3s 668us/step - loss: 0.3548 - accuracy: 0.8427
Epoch 4/50
4514/4514 [==============================] - 3s 676us/step - loss: 0.3541 - accuracy: 0.8432
Epoch 5/50
4514/4514 [==============================] - 3s 671us/step - loss: 0.3539 - accuracy: 0.8421
Epoch 6/50
4514/4514 [==============================] - 3s 668us/step - loss: 0.3531 - accuracy: 0.8427
Epoch 7/50
4514/4514 [==============================] - 3s 676us/step - loss: 0.3526 - accuracy: 0.8431
Epoch 8/50
4514/4514 [==============================] - 3s 672us/step - loss: 0.3521 - accuracy: 0.8431
Epoch 9/50
4514/4514 [==============================] - 3s 672us/step - loss: 0.3512 - accuracy: 0.8444
Epoch 10/50
4514/4514 [==============================] - 3s 675u

### Best Hyperparameters

Imprimimos la combinación de los parámetros y sus resultados, obtenemos la mejor combinación de estos para obtener el mejor resultado.


In [38]:
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.844714 using {'init_mode': 'normal'}
0.843628 (0.003593) with: {'init_mode': 'uniform'}
0.843628 (0.002541) with: {'init_mode': 'lecun_uniform'}
0.844714 (0.002108) with: {'init_mode': 'normal'}
0.778802 (0.002084) with: {'init_mode': 'zero'}
0.844536 (0.003359) with: {'init_mode': 'glorot_normal'}
0.844226 (0.003268) with: {'init_mode': 'glorot_uniform'}
0.843318 (0.001267) with: {'init_mode': 'he_normal'}
0.844005 (0.003011) with: {'init_mode': 'he_uniform'}


In [26]:
# Function to create model,for KerasClassifier
def create_best_activation_model():
    #defining my model
    mymodel = Sequential()
    mymodel.add(Dense(9,kernel_initializer = 'normal', input_dim=9, activation='relu'))
    mymodel.add(Dense(27,kernel_initializer = 'normal', activation='relu'))
    mymodel.add(Dense(12,kernel_initializer = 'normal', activation='relu'))
    mymodel.add(Dense(1,kernel_initializer = 'normal', activation='sigmoid'))
    
    # Compile the model
    mymodel.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return mymodel

In [27]:
activation_model = KerasClassifier(build_fn=create_best_activation_model, epochs=50, batch_size=10)

In [28]:
y = data.RainTomorrow
kfold = KFold(n_splits=10, shuffle=True, random_state=random.randint(0, 10)) 
results = cross_val_score(activation_model, x, y, cv=kfold, n_jobs=-1)
print(results.mean())                    

0.8466324090957642


## Resumen

En síntesis una vez aplicado el dataset para predecir la lluvia en un dataset en una red neuronal compleja 2 capas ocultas con 27 y 12 neuronas. Entre los parámetros se escogieron los siguientes: Batch_size, Epoch, Activation function.
De los cuales obtuvimos como mejores la siguiente combinación:

* Batch_size = 10
* Epoch = 50
* Activation function = normal

Con esta combinación de parámetros obtuvimos una precisión de 0.8466324090957642.


# Red neuronal simple
Luego de haber realizado una red neuronal compleja en su arquitectura, realizamos una red neuronal más simple; con una capa oculta y distintas cantidades de neuronas.
La capa de entrada, contiene 9 neuronas,una por cada parámetro. 

Usaré otra regla que establece que la cantidad de neuronas ocultas debe ser igual a 2/3 de la suma de las neuronas en la capa de entrada más las neuronas en la capa de salida. Por lo que la cantidad de neuronas es $\dfrac{2}{3}\cdot 10 = 6 $.

En síntesis, esta red neuronal tiene solamente una capa oculta con 6 neuronas.


## Prueba de parámetros

## Batch size y epochs

En esta sección estaremos probando con distintas cantidades de batch y epochs, para averiguar cual es la que tiene mejores resultados.




In [29]:
# Function to create model,for KerasClassifier
def create_my_model():
    #defining my model
    mymodel = Sequential()
    mymodel.add(Dense(9, input_dim=9, activation='relu'))
    mymodel.add(Dense(6, activation='relu'))
    mymodel.add(Dense(1, activation='sigmoid'))
    
    # Compile the model
    mymodel.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return mymodel

### Creación  del modelo

In [43]:
# create model
model = KerasClassifier(build_fn=create_my_model)

De igual manera se realizará una comprobación de parámetros, esta vez con más variedad. Dado que la red es más pequeña, mis recursos computacionales lo permiten. 
 


In [44]:
# define the grid search parameters
batchSize = [10, 20, 40, 60, 80, 100]
epochs = [10, 30, 50]
grid = dict(batch_size=batchSize, epochs=epochs)

### Tuning  Hyperparameters

 Se ejecuta la combinación de cada parámetro en un stratified  cross validation.

In [45]:

cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
grid_search = GridSearchCV(estimator=model, param_grid=grid, n_jobs=-1, cv=cv, scoring='accuracy',error_score=0)
grid_result = grid_search.fit(train_x, train_y)

Epoch 1/30
1129/1129 [==============================] - 1s 629us/step - loss: 0.4106 - accuracy: 0.8053
Epoch 2/30
1129/1129 [==============================] - 1s 629us/step - loss: 0.3688 - accuracy: 0.8326
Epoch 3/30
1129/1129 [==============================] - 1s 627us/step - loss: 0.3589 - accuracy: 0.8402
Epoch 4/30
1129/1129 [==============================] - 1s 624us/step - loss: 0.3550 - accuracy: 0.8427
Epoch 5/30
1129/1129 [==============================] - 1s 624us/step - loss: 0.3531 - accuracy: 0.8448
Epoch 6/30
1129/1129 [==============================] - 1s 622us/step - loss: 0.3520 - accuracy: 0.8444
Epoch 7/30
1129/1129 [==============================] - 1s 628us/step - loss: 0.3513 - accuracy: 0.8448
Epoch 8/30
1129/1129 [==============================] - 1s 628us/step - loss: 0.3505 - accuracy: 0.8455
Epoch 9/30
1129/1129 [==============================] - 1s 625us/step - loss: 0.3498 - accuracy: 0.8458
Epoch 10/30
1129/1129 [==============================] - 1s 630u

### Best Hyperparameters
 
Imprimimos la combinación de los parámetros y sus resultados, obtenemos la mejor combinación de estos para obtener el mejor resultado.
 



In [46]:
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.846073 using {'batch_size': 40, 'epochs': 30}
0.844197 (0.004558) with: {'batch_size': 10, 'epochs': 10}
0.845733 (0.004302) with: {'batch_size': 10, 'epochs': 30}
0.845696 (0.004375) with: {'batch_size': 10, 'epochs': 50}
0.844647 (0.004761) with: {'batch_size': 20, 'epochs': 10}
0.845482 (0.004294) with: {'batch_size': 20, 'epochs': 30}
0.845600 (0.005154) with: {'batch_size': 20, 'epochs': 50}
0.843960 (0.004504) with: {'batch_size': 40, 'epochs': 10}
0.846073 (0.004799) with: {'batch_size': 40, 'epochs': 30}
0.845762 (0.005228) with: {'batch_size': 40, 'epochs': 50}
0.844079 (0.004566) with: {'batch_size': 60, 'epochs': 10}
0.845164 (0.004469) with: {'batch_size': 60, 'epochs': 30}
0.845408 (0.004377) with: {'batch_size': 60, 'epochs': 50}
0.843539 (0.004422) with: {'batch_size': 80, 'epochs': 10}
0.844854 (0.004157) with: {'batch_size': 80, 'epochs': 30}
0.845607 (0.004265) with: {'batch_size': 80, 'epochs': 50}
0.843384 (0.005036) with: {'batch_size': 100, 'epochs': 10}
0

In [30]:
# create model
model = KerasClassifier(build_fn=create_my_model, epochs=30, batch_size=40)

In [31]:
y = data.RainTomorrow
kfold = KFold(n_splits=10, shuffle=True, random_state=random.randint(0, 10)) 
results = cross_val_score(model, x, y, cv=kfold, n_jobs=-1)
print(results.mean())           

0.846242469549179


## Optimizador de la función de activación
 
En esta parte probaremos el optimizador, de esta manera conoceremos con cual es que tenemos mejor precisión en el modelo.
 



In [32]:
def create_model_optimizer(optimizer='adam'):
    #defining my model
    mymodel = Sequential()
    mymodel.add(Dense(9, input_dim=9, activation='relu'))
    mymodel.add(Dense(6, activation='relu'))
    mymodel.add(Dense(1, activation='sigmoid'))
    # Compile model
    mymodel.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return mymodel

In [50]:
model_opt = KerasClassifier(build_fn=create_model_optimizer, epochs=30, batch_size=40)

### Parameters



In [51]:
# define the grid search parameters
optimizer = ['SGD','Adadelta', 'RMSprop', 'Adagrad','Adam']

grid = dict(optimizer=optimizer)

### Tuning Hyperparameters

 Se ejecuta la combinación de cada parámetro en un stratified cross validation.

In [52]:

cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
grid_search = GridSearchCV(estimator=model_opt, param_grid=grid, n_jobs=-1, cv=cv, scoring='accuracy',error_score=0)
grid_result = grid_search.fit(train_x, train_y)

Epoch 1/30
1129/1129 [==============================] - 1s 653us/step - loss: 0.4381 - accuracy: 0.8043
Epoch 2/30
1129/1129 [==============================] - 1s 648us/step - loss: 0.3603 - accuracy: 0.8386
Epoch 3/30
1129/1129 [==============================] - 1s 657us/step - loss: 0.3547 - accuracy: 0.8423
Epoch 4/30
1129/1129 [==============================] - 1s 644us/step - loss: 0.3524 - accuracy: 0.8436
Epoch 5/30
1129/1129 [==============================] - 1s 631us/step - loss: 0.3512 - accuracy: 0.8438
Epoch 6/30
1129/1129 [==============================] - 1s 639us/step - loss: 0.3506 - accuracy: 0.8447
Epoch 7/30
1129/1129 [==============================] - 1s 634us/step - loss: 0.3501 - accuracy: 0.8449
Epoch 8/30
1129/1129 [==============================] - 1s 638us/step - loss: 0.3498 - accuracy: 0.8445
Epoch 9/30
1129/1129 [==============================] - 1s 630us/step - loss: 0.3493 - accuracy: 0.8452
Epoch 10/30
1129/1129 [==============================] - 1s 640u

### Best Hyperparameters

Imprimimos la combinación de los parámetros y sus resultados, obtenemos la mejor combinacion de estos para obtener el mejor resultado.

In [53]:
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.845570 using {'optimizer': 'Adam'}
0.844056 (0.004267) with: {'optimizer': 'SGD'}
0.687022 (0.090215) with: {'optimizer': 'Adadelta'}
0.845400 (0.004650) with: {'optimizer': 'RMSprop'}
0.822447 (0.014578) with: {'optimizer': 'Adagrad'}
0.845570 (0.004621) with: {'optimizer': 'Adam'}


### Creación del modelo con los mejores parámetros



In [33]:
model = KerasClassifier(build_fn=create_model_optimizer, epochs=30, batch_size=40)

## Training with hyperparameters
En esta parte se prueba el modelo con los mejores hiperparametros, con distintos datos. Esto con el propósito de evitar algún sesgo generado en el split. Razón por la que se usa un número aleatorio  para el random_state, de esta manera obtendremos distintas secciones de training y test.


In [34]:
y = data.RainTomorrow
kfold = KFold(n_splits=10, shuffle=True, random_state=random.randint(0, 10)) 
results = cross_val_score(model, x, y, cv=kfold, n_jobs=-1)
print(results.mean())

0.8469337165355683


0.8469337165355683## Resumen
    mymodel.add(Dense(9, input_dim=9, activation='relu'))
    mymodel.add(Dense(6, activation='relu'))
    mymodel.add(Dense(1, activation='sigmoid'))
En síntesis una vez aplicado el dataset para predecir la lluvia en un dataset en una red neuronal mas simple, con una capa oculta con 6 neuronas. Entre los parámetros se escogieron los siguientes: Batch_size, Epoch, Activation function.
De los cuales obtuvimos como mejores la siguiente combinación:

* Batch_size = 40
* Epoch = 30
* Activation function optimizer = Adam

Con esta combinación se obtuvo una precisión de 0.8469337165355683, muy similar a la obtenida en la red neuronal más compleja.